In [1]:
from seisLM.model import lit_model
from seisLM.model import multidim_wav2vec2
import seisbench.data as sbd
import torch

In [2]:
data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms).unsqueeze(0).cuda()



2024-06-02 19:50:26,204 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [3]:
pretrained_model = lit_model.LitMultiDimWav2Vec2.load_from_checkpoint(
  '/home/liu0003/Desktop/projects/seisLM/saved_models/seisLM/42__2024-05-26-17h-04m-04s/checkpoints/epoch=3-step=218924.ckpt'
).model

In [4]:
config = pretrained_model.config
config.num_labels = 3
classification_model = multidim_wav2vec2.MultiDimWav2Vec2ForFrameClassification(config)

# Copy weights from pretrained model (if needed)
classification_model.wav2vec2.load_state_dict(pretrained_model.wav2vec2.state_dict())

classification_model.cuda()


MultiDimWav2Vec2ForFrameClassification(
  (wav2vec2): MultiDimWav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(3, 256, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(256, 256, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=256, out_features=240, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Wav2Vec2EncoderStableLayerNorm(
      (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(


In [5]:
import torch

# Define the shape
shape = (1, 1499, 3)

# Create an empty tensor of zeros
one_hot_tensor = torch.zeros(shape)

# Randomly select indices along the last dimension to be set to 1
random_indices = torch.randint(0, shape[-1], size=shape[:-1])

# Use advanced indexing to set the appropriate elements to 1
one_hot_tensor[torch.arange(shape[0])[:, None], torch.arange(shape[1]), random_indices] = 1

print(one_hot_tensor)


tensor([[[0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         ...,
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]])


In [8]:
outputs = classification_model(
  input_values=input_values,
  labels=one_hot_tensor.cuda() #torch.argmax(one_hot_tensor, 1).cuda()
)

reshaped_label torch.Size([1499])
reshaped_logits torch.Size([1499, 3])


TokenClassifierOutput(loss=tensor(1.1484, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[-0.2018,  0.2585, -0.4875],
         [-0.0398,  0.0275, -0.2729],
         [ 0.0502, -0.0387, -0.1743],
         ...,
         [ 0.5099,  0.1248, -0.5599],
         [ 0.2934, -0.0410, -0.4462],
         [ 0.2642,  0.4334, -0.7996]]], device='cuda:0',
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [9]:
outputs = classification_model(
  input_values=input_values,
  labels=torch.argmax(one_hot_tensor, 1).cuda()
)


outputs

reshaped_label torch.Size([1])
reshaped_logits torch.Size([1499, 3])


ValueError: Expected input batch_size (1499) to match target batch_size (1).

In [ ]:
# labels = torch.argmax(outputs[0], 2)
# torch.argmax(labels, 1)



torch.Size([1499])